In [7]:
import logging

from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

exe = Transformation(
        "checkpoint_program.py",
        site="local",
        pfn=str(Path(".") / "checkpoint_program.py"),
        is_stageable=True
    )

tc = TransformationCatalog().add_transformations(exe).write()

job = Job(exe)\
        .add_checkpoint(File("saved_state.txt"), stage_out=True)\
        .add_pegasus_profile(checkpoint_time=1, max_walltime=2)

wf = Workflow("checkpoint-wf").add_jobs(job)

try:
    wf.plan(submit=True).wait().analyze()
except PegasusClientError as e:
    print(e.output)

2020.07.02 05:09:48.318 UTC: [INFO]  Planner launched in the following directory /home/scitech/shared-data/sample-checkpoint-wf 
2020.07.02 05:09:48.323 UTC: [INFO]  Planner invoked with following arguments --output-sites local --cleanup inplace -vvv --submit --dax workflow.yml  
2020.07.02 05:09:48.325 UTC: [CONFIG]  Pegasus Properties set by the user 
2020.07.02 05:09:48.325 UTC: [CONFIG]  pegasus.home.bindir=/home/scitech/pegasus/dist/pegasus-5.0.0dev/bin 
2020.07.02 05:09:48.326 UTC: [CONFIG]  pegasus.home.schemadir=/home/scitech/pegasus/dist/pegasus-5.0.0dev/share/pegasus/schema 
2020.07.02 05:09:48.326 UTC: [CONFIG]  pegasus.home.sharedstatedir=/home/scitech/pegasus/dist/pegasus-5.0.0dev/share/pegasus 
2020.07.02 05:09:48.326 UTC: [CONFIG]  pegasus.home.sysconfdir=/home/scitech/pegasus/dist/pegasus-5.0.0dev/etc 
ERROR while logging metrics The metrics file location is not yet initialized
2020.07.02 05:09:48.738 UTC: [DEBUG]  Exiting with non-zero exit-code 1 
2020.07.02 05:09:48.